### RECODE LOOKUP JENIS KEDIAMAN

In [1]:
import pandas as pd

load data

In [2]:
#print(pd.read_csv(r'\\10.21.45.179\mySPC\JKPD LMI\Cleansed For CTADR\20221129\MPIC_MTIB_3PROFESSIONAL MANAGEMENT DEVELOPMENT PROGRAM_20221129_28.csv',sep='|'))
recode = pd.read_csv(r"\\10.21.45.172\padu_sharing\EKUITI&ASET\RECODE_RUMAH.csv", encoding='latin', low_memory=False)
recode.head()

,JPN_ID,JENIS_KEDIAMAN
0,30303134383830338216126030353031,SEMI-D
1,30303136383830335918588032313035,RUMAH TERES 1 TINGKAT
2,30303238373231327540924030323132,"LOT NO.49,.TAMAN ADIKA,PHASE 4,.KG.BARIAWA KEN..."
3,30303238373530388723802031393039,SEBIDANG TANAH KAMPUNG.
4,30303539383630364489694032343233,RUMAH SEMI-D SETINGKAT


In [3]:
# !pip install pyocclient -qq

lookup table/file

In [4]:
import pandas as pd
import owncloud

# Replace with the actual public link and folder password
public_link = 'https://cloud.stats.gov.my/index.php/s/Bhimkzhhr4J2pcH' ##gciOpJBOv4Fnw4j
folder_password = ''

# Initialize the OwnCloud client
oc = owncloud.Client.from_public_link(public_link, folder_password=folder_password)

###================================ list of files

# files = oc.list('/')

# for file in files:
#     if not file.is_dir():
#         print(file.get_name())

##================================ sheet in files

# # Define the path to the Excel file
# file_path = '/LIST OF VARIABLES AND LOOK UP V12.xlsx'

# # Download the file from OwnCloud
# file_content = oc.get_file_contents(file_path)

# # # Create a Pandas DataFrame from the Excel file
# df = pd.read_excel(file_content, sheet_name=None)

# # Get the sheet names as a list
# sheet_names = list(df.keys())

# # Print the sheet names to verify that the list was obtained correctly
# print("Sheet names:")
# for sheet in sheet_names:
#     print(sheet)

##================================ read data in sheet

# Define the path to the Excel file
file_path = '/LIST OF VARIABLES AND LOOK UP V12.xlsx'

# Download the file from OwnCloud
file_content = oc.get_file_contents(file_path)

df = pd.read_excel(file_content, sheet_name='JENIS_TEMPAT_KEDIAMAN')
# Print the first few rows of the dataframe
vlook_jkediaman = df.iloc[:-1,:]
vlook_jkediaman


,id,nama
0,1,SESEBUAH
1,2,BERKEMBAR
2,3,TERES
3,4,BERANGKAI
4,5,BERKELOMPOK
5,6,RUMAH BANDAR
6,7,RUMAH PANGSA
7,8,PANGSAPURI
8,9,KONDOMINIUM
9,10,PANGSAPURI KHIDMAT


`recode : nan/tiada maklumat to blank`

matching data with lookup table

In [5]:
import pandas as pd

df_data = recode
df_dict = vlook_jkediaman

# Function to map JENIS_KEDIAMAN to id
def map_to_id(jenis_kediaman):
    if isinstance(jenis_kediaman, str):
        for name, id in zip(df_dict['nama'], df_dict['id']):
            if name in jenis_kediaman:
                return id
    return None

# Map JENIS_KEDIAMAN to id and create a new 'id' column
df_data['id'] = df_data['JENIS_KEDIAMAN'].apply(map_to_id)

df_data



,JPN_ID,JENIS_KEDIAMAN,id
0,30303134383830338216126030353031,SEMI-D,NaN
1,30303136383830335918588032313035,RUMAH TERES 1 TINGKAT,3.0
2,30303238373231327540924030323132,"LOT NO.49,.TAMAN ADIKA,PHASE 4,.KG.BARIAWA KEN...",NaN
3,30303238373530388723802031393039,SEBIDANG TANAH KAMPUNG.,NaN
4,30303539383630364489694032343233,RUMAH SEMI-D SETINGKAT,NaN
...,...,...,...
618529,39393430363830314691041030313038,RUMAH TERES 2 TINGKAT,3.0
618530,39393434373530319176898030313033,TERES 2 1/2 TINGKAT,3.0
618531,39393638373830316608970303131310,RUMAH TERES SATU TINGKAT,3.0
618532,39393835363630316900563030313038,RUMAH,NaN


summaries result

In [6]:
# Group by 'id' and count the occurrences, then merge with df_dict to include 'nama'
grouped = df_data.groupby('id')['JPN_ID'].size().reset_index(name='count')
result = pd.merge(grouped, df_dict, on='id')
result[['id','nama','count']]

# Calculate the sum of counts
total_count = result['count'].sum()

print(result)
print("")
# print("Total Success Recode:", total_count)
# print("Total Nan:", (df_data.shape[0]-total_count))
print("Total Records:", f"{df_data.shape[0]:,}")
print("Total Success Recode:", f"{total_count:,}")
print("Total NaN:", f"{df_data.shape[0] - total_count:,}")

      id   count               nama
0    1.0    2235           SESEBUAH
1    2.0   29228          BERKEMBAR
2    3.0  242015              TERES
3    4.0      87          BERANGKAI
4    5.0      43        BERKELOMPOK
5    6.0     249       RUMAH BANDAR
6    7.0   10550       RUMAH PANGSA
7    8.0    9580         PANGSAPURI
8    9.0    4865        KONDOMINIUM
9   13.0      28     RUMAH PANJANG 
10  15.0    2977              BILIK
11  16.0       1  RUMAH KAMPUNG AIR

Total Records: 618,534
Total Success Recode: 301,858
Total NaN: 316,676


count for successful match records

In [7]:
df_data[df_data['id'].notna()].shape

(301858, 3)

save to csv

In [8]:
# df_data.to_csv('RECODE_RUMAH_DONE.csv', index=False)

data checking

In [9]:
df_data.head()

,JPN_ID,JENIS_KEDIAMAN,id
0,30303134383830338216126030353031,SEMI-D,NaN
1,30303136383830335918588032313035,RUMAH TERES 1 TINGKAT,3.0
2,30303238373231327540924030323132,"LOT NO.49,.TAMAN ADIKA,PHASE 4,.KG.BARIAWA KEN...",NaN
3,30303238373530388723802031393039,SEBIDANG TANAH KAMPUNG.,NaN
4,30303539383630364489694032343233,RUMAH SEMI-D SETINGKAT,NaN


successful match records

In [10]:
df_data[df_data['id']==1.0]

,JPN_ID,JENIS_KEDIAMAN,id
58,35303030363131309291084032313038,RUMAH SESEBUAH,1.0
328,35303030363730372555427031343032,SESEBUAH,1.0
595,35303030373330352634303030393037,RUMAH SESEBUAH/ BANGLO SETINGKAT,1.0
767,35303030373830364025253030363032,RUMAH BANGLO/SESEBUAH,1.0
933,35303030383130332218198030343132,RUMAH SESEBUAH,1.0
...,...,...,...
617478,38303939373930319483422030313038,RUMAH SESEBUAH,1.0
618028,38353633363530316157451030323038,SESEBUAH (KG CHANGKAT LARANG),1.0
618054,38363035363930317354951030313038,RUMAH SESEBUAH (TAMAN MERU),1.0
618141,38373236373130314842185030313033,SATU UNIT RUMAH KEDIAMAN SESEBUAH DATAS LOT 77...,1.0


In [11]:
df_data[df_data['id']==13.0].head()

,JPN_ID,JENIS_KEDIAMAN,id
11189,35303133363830357090162032343133,RUMAH PANJANG 2 TKT,13.0
99901,35313238373931302676715032313133,PROGRAM PINJAMAN RUMAH PANJANG DARI PERBADANAN...,13.0
138302,35313833353930349500256031323133,RUMAH PANJANG - SEMI KONKRIT.,13.0
164853,35323231373031324710098031303133,RUMAH PANJANG TURUN TEMURUN.,13.0
175873,35323338363530378078768030373133,RUMAH PANJANG 2 TINGKAT,13.0


count for unsuccessful match records

In [12]:
df_data[df_data['id'].isna()].shape

(316676, 3)

example for unsuccessful match records

In [13]:
df_data[df_data['id'].isna()].head(10)

,JPN_ID,JENIS_KEDIAMAN,id
0,30303134383830338216126030353031,SEMI-D,NaN
2,30303238373231327540924030323132,"LOT NO.49,.TAMAN ADIKA,PHASE 4,.KG.BARIAWA KEN...",NaN
3,30303238373530388723802031393039,SEBIDANG TANAH KAMPUNG.,NaN
4,30303539383630364489694032343233,RUMAH SEMI-D SETINGKAT,NaN
5,35303030353930316832220313330330,SEBUAH RUMAH BANGLO SATU TINGKAT DIATAS TANAH ...,NaN
6,35303030353930322997950323030320,RUMAH LOT,NaN
9,35303030353930357979340313830370,SEMI-D,NaN
10,35303030353930375949036030383133,TERRACED HOUSE,NaN
11,35303030353930379036270303330330,BANGLO 1 TINGKAT,NaN
12,35303030353931308847535030313131,"P.T.1697, PEMIN JAYA III, CHENERING, KUALA TER...",NaN


In [14]:
df_data[df_data['id'].isna()].iloc[11:21]

,JPN_ID,JENIS_KEDIAMAN,id
14,35303030353931308847535030313131,RUMAH SETINGKAT,NaN
15,35303030353931312597915031323038,APARTMENT KOS RENDAH,NaN
16,35303030353931312597915031323038,LOW-MEDIUM COST APARTMENT,NaN
17,35303030353931312597915031323038,RUMAH KOS RENDAH,NaN
19,35303030353931314659313031363131,KEDIAMAN,NaN
22,35303030363030319218502032333033,RUMAH BANGLO,NaN
27,35303030363030331710309031383033,RUMAH KEDIAMAN DUA TINGKAT,NaN
29,35303030363030352651011030383031,APARTMENT.HAK MILIK STRATA,NaN
30,35303030363030385199210313930370,RUMAH DUA TINGKAT SEMI-DETACHED,NaN
32,35303030363031301915525030353036,MEMBINA RUMAH DIATAS TANAH,NaN


In [15]:
df_data[df_data['id'].isna()].iloc[22:32]

,JPN_ID,JENIS_KEDIAMAN,id
34,35303030363031301915525030353036,RUMAH KEDIAMAN,NaN
35,35303030363031302127325033313039,RUMAH KAMPUNG,NaN
36,35303030363031316573426032363033,RUMAH KOS RENDAH,NaN
38,35303030363031318598040032353037,RUMAH APARTMEN KOS SEDERHANA,NaN
45,35303030363130317908512030383031,APARTMENT KOS SEDERHANA,NaN
48,35303030363130357707008032393035,RUMAH SEBUAH DI ATAS TANAH LOT YANG DIBELI.,NaN
50,35303030363130396897804030333131,SPNB,NaN
52,35303030363131306909266031373131,TANAH DAN RUMAH SEMID SATU TINGKAT,NaN
53,35303030363131307846803031303200,4-STOREY APARTMENT (IT CENTRE POINT),NaN
55,35303030363131309291084032313038,BANDAR SRI SUBANG DAMAI APARTMENT,NaN


In [16]:
df_data[df_data['id'].isna()].tail(10)

,JPN_ID,JENIS_KEDIAMAN,id
618514,39373938363930314235277030313038,"HOUSE TYPE :2-H, LOT. NO: 1593, INTERMEDIATE.T...",NaN
618515,39373938363930314235277030313038,RUMAH SEMI-D SINGLE STORY,NaN
618518,39383139363930315703743030313038,"LOT 240 DENGKIL, SELANGOR",NaN
618520,39383239363430311847046030313130,PERSENDIRIAN,NaN
618521,39383239363430311847046030313130,PERSENDIRIAN.,NaN
618522,39383337373830318147341030313033,NO RUJUKAN SPPB / NO FAIL PINJAMAN\t:\t60827/2...,NaN
618523,39383437373530318904605030313031,RUMAH KOS RENDAH SEDERHANA,NaN
618528,39393337373130311450142030313038,SEMI D,NaN
618532,39393835363630316900563030313038,RUMAH,NaN
618533,39393835363630316900563030313038,RUMAH PUTRAJAYA PRESINT 15,NaN
